In [1]:
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import os
import time

import matplotlib.pyplot as plt
import seaborn as sns
import folium as fl
from folium import plugins
import geopandas as gpd
import webbrowser

from Priorizacion_funciones.heavyusers_funs import *

from sklearn.cluster import DBSCAN, KMeans, MeanShift
os.getcwd()

'C:\\Users\\DarioAlejandroLatorr\\OneDrive\\claro_20_abril_2021\\ENVIO_DARIO\\TRONCALES_HEAVYUSERS_DATAGEO'

# MANZANAS CON NOMBRE DE MUNICIPIO/CIUDAD

La siguiente línea no es necesaria correrla si ya existe el archivo MGN_URB_MANZANA_NOM_MUNICIPIO.shp

In [3]:
shap_manzana_nom_mun =  gpd.read_file('Data_geo\MANZANA\MGN_URB_MANZANA.shp')
shap_politico_nom_mun =  gpd.read_file('Data_geo\ADMINISTRATIVO\MGN_MPIO_POLITICO.shp')[['DPTO_CCDGO','MPIO_CCDGO','MPIO_CNMBR']]
shap_politico_nom_mun['COD_MPIO'] = shap_politico_nom_mun['DPTO_CCDGO'] + shap_politico_nom_mun['MPIO_CCDGO']
shap_manzana_nom_mun = shap_manzana_nom_mun.merge(shap_politico_nom_mun, on='COD_MPIO', how='left')

dic_vocales = {'Á':'A','É':'E','Í':'I','Ó':'O','Ú':'U'}
def quit_tildes(ciudad):
    for voc_til in dic_vocales.keys():
        ciudad = ciudad.replace(voc_til,dic_vocales[voc_til])
    return ciudad
shap_manzana_nom_mun['CIUDAD'] = [quit_tildes(ciudad) for ciudad in  shap_manzana_nom_mun['MPIO_CNMBR']]

shap_manzana_nom_mun.to_file('Data_geo\MANZANA\MGN_URB_MANZANA_NOM_MUNICIPIO.shp')

In [3]:
t = time.time()
name_file = 'USERS_CLASIFICADOS_COMPLETO.CSV'
list_columns =['CUENTA','CUENTA_MATRIZ', 'NODO',
                   'CIUDAD', 'LATITUD','LONGITUD', 
                   'flbb_down_traffic_bytes','flbb_up_traffic_bytes',
                   'traf_total']
path_he_us = f'Datos_heavy_users/{name_file}'
path_manzanas = 'Data_geo\MANZANA\MGN_URB_MANZANA_NOM_MUNICIPIO.shp'
    
df_users = pd.read_csv(path_he_us, sep='|')[list_columns]
shape_manzanas = gpd.read_file(path_manzanas)[['CIUDAD','geometry']]
time.time() -t

81.4844560623169

In [4]:
t = time.time()
path_ = 'Data_geo/HEAVYUSERS_ANALISIS/'
ciudad = 'PASTO'
# todos los nodos de la ciudad
nodos_ciudad = df_users.loc[df_users['CIUDAD']==ciudad,'NODO'].unique()
# nodos de prueba
nodos_ciudad = ['PCN','JDS','LCD']
lim_coord_atipica = 0.03 # aprox 3 km
percentil = 90  # de momento se consideran heavy users a los usuarios con el tráfico total superior al del percentil
# ancho de banda relacionado con las distancias 
ban_list = np.arange(0.0006,0.0010,0.0001)
# 0.0009  aprox 100 metros https://www.usna.edu/Users/oceano/pguth/md_help/html/approx_equivalents.htm

# limite de usuarios los valores pueden ser modificados el límite es l número de puertos aunque según indicaciones siempre
# se dejan algunos libres para nuevos clientes.
lim_usuarios_list = np.array([32,64,100])
incluir_cuentas_matriz = True
incluir_cuentas_horizontales = False

#######################################################################################
list_ari_cuadra = ['LONGITUD_CENTER_ARISTA_CUADRA','LATITUD_CENTER_ARISTA_CUADRA']
colors_list = ['lightblue', 'red', 'black', 'green',  'blue', 'orange','darkblue',
                       'cadetblue', 'pink', 'lightgreen', 'gray', 'darkred', 'purple',
                       'lightblue', 'red', 'black', 'green',  'blue', 'orange','darkblue',
                       'cadetblue', 'pink', 'lightgreen', 'gray', 'darkred', 'purple']
create_dir(f'{path_}{ciudad}')
# listas para dataframe info
for nodo in nodos_ciudad:
    try:
        # alistando capas, guardando archivos 

        df_usuarios, shape_ciudad = get_data(df_users, shape_manzanas, ciudad, nodo,
                                                     incluir_cuentas_matriz=incluir_cuentas_matriz,
                                                     incluir_cuentas_horizontales=incluir_cuentas_horizontales)
        l_df_usuarios_ini = len(df_usuarios)

        df_usuarios_analisis = drop_users_far_center(df_usuarios, lim_atipica = lim_coord_atipica)
        df_usuarios_analisis = prep_and_save_datageo(df_usuarios_analisis, shape_ciudad, path_, ciudad, nodo)

        trafi_total_usuarios_analisis = df_usuarios_analisis["traf_total"].sum()
        
        # seleccionando a los heavyusers
        hu_treshold = np.percentile(df_usuarios_analisis['traf_total'], percentil)
        df_hu = df_usuarios_analisis[df_usuarios_analisis['traf_total']>=hu_treshold].copy()

        #guardando archivos
        path_nodo = f'{path_}/{ciudad}/{nodo}'
        save_csv(df_usuarios, f'{path_nodo}/all_users.csv')
        df_usuarios_analisis_pre_pro = df_usuarios_analisis.copy()
        save_csv(df_hu, f'{path_nodo}/heavy_users.csv')


        # creando y guardando mapa de los usuarios para el análisis.
        df_no_hu = df_usuarios_analisis[df_usuarios_analisis['traf_total']<hu_treshold]
        create_map_users(df_no_hu, df_hu, path_nodo)
        for ban in ban_list:
            for lim_usuarios in lim_usuarios_list:
                
                path_folder = f'{path_nodo}/dist_aprox_{int(ban*100/0.0009)}_lim_usuarios_{lim_usuarios}'
                create_dir(path_folder)


                # haciendo clusters
                clu_meanshift = MeanShift(bandwidth=ban, cluster_all=False)
                ## el ajuste se hace con los heavyusers
                X = df_hu[list_ari_cuadra]
                clu_meanshift.fit(X)
                ## la predicción se hace con todos los usuarios del análisis
                X_pred = df_usuarios_analisis[list_ari_cuadra].values
                df_usuarios_analisis['CLUSTER'] = clu_meanshift.predict(X_pred)

                # excluyendo usuarios que no fueron clusterizados
                mask_con_cluster = (df_usuarios_analisis['CLUSTER']!=-1)

                df_usuarios_analisis_clusterizados = df_usuarios_analisis[mask_con_cluster].copy()

                # rankenado usuarios y escogiendo para los clusters
                df_usuarios_analisis_clusterizados['rank'] = df_usuarios_analisis_clusterizados.groupby('CLUSTER')["traf_total"].rank(ascending=False)
                mask_usuarios_max = (df_usuarios_analisis_clusterizados['rank']<=lim_usuarios)
                df_usuarios_analisis_clusterizados = df_usuarios_analisis_clusterizados[mask_usuarios_max].copy()

                # guardando información del modelo y de las cuentas 

                df_usuarios_analisis_clu_noclu = df_usuarios_analisis_pre_pro.merge(df_usuarios_analisis_clusterizados[['CUENTA','CLUSTER']],
                                                                                 on='CUENTA', how='left')
                df_usuarios_analisis_clu_noclu.fillna(value={'CLUSTER':-2}, inplace=True)

                df_group = df_usuarios_analisis_clu_noclu.groupby(['CUENTA_MATRIZ','CLUSTER']).agg({'CUENTA':'count','traf_total':'sum'}).reset_index()

                save_csv(df_usuarios_analisis_clu_noclu, f'{path_folder}/analysis_users.csv')
                save_csv(df_group, f'{path_folder}/resumen_modelo.csv')

                df_centroides = create_map_users_clusters(df_usuarios_analisis_clusterizados, path_folder, colors_list)
                save_csv(df_centroides,  f'{path_folder}/centroides.csv')
            
    except:
        pass
time.time() -t

28.337045669555664